In [1]:
import torch
from torch import tensor


We now illustrate implementing matrix multiplication using torch operations on torch tensors. First, we use an ad-hoc implementation of the dot product.

In [2]:
def matmul(a, b):
    (ar, ac), (br, bc) = a.shape, b.shape
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            c[i, j] = (a[i, :] * b[:, j]).sum()
    return c


We setup our test data.

In [3]:
from fastai.vision.all import *

pickle_path = URLs.path('mnist_png')/'mnist_png.pkl'
path = untar_data(URLs.MNIST)/'training'

if not pickle_path.exists():
    pickle_path.parent.mkdir(parents=True, exist_ok=True)
    ds = DataBlock(
        blocks = (ImageBlock(PILImageBW), CategoryBlock),
        get_items = get_image_files,
        get_y = parent_label,
        splitter = RandomSplitter(1/6, seed=0)
    ).datasets(path)

    xs, ys = zip(*ds.train, *ds.valid)
    xs = np.stack(L(map(lambda x: np.array(x, dtype=np.float32).reshape(-1), xs))) / 255.
    ys = np.array(ys, dtype=np.int64)

    x_train, x_valid = xs[:len(ds.train)], xs[len(ds.train):]
    y_train, y_valid = ys[:len(ds.train)], ys[len(ds.train):]

    save_pickle(pickle_path, [x_train, y_train, x_valid, y_valid])

    del ds, xs, ys, x_train, y_train, x_valid, y_valid

x_train, y_train, x_valid, y_valid = map(tensor, load_pickle(pickle_path))

torch.manual_seed(1)
weights = torch.randn(784, 10)
bias = torch.zeros(10)

m1 = x_valid[:5]
m2 = weights
ar, ac = m1.shape 
br, bc = m2.shape

t1 = torch.zeros(ar, bc)

for i in range(ar):         # 5
    for j in range(bc):     # 10
        for k in range(ac): # 784
            t1[i, j] += m1[i, k] * m2[k, j]


Verify correctness.

In [4]:
test_close(t1, matmul(m1, m2))


We see that torch on CPU with the ad-hoc dot product is slightly slower than `njit`.

In [5]:
%timeit -n 50 _ = matmul(m1, m2)


590 µs ± 39.5 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)


We now consider using PyTorch's dot product subroutine.

In [6]:
def matmul(a, b):
    (ar, ac), (br, bc) = a.shape, b.shape
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            c[i, j] = torch.dot(a[i, :], b[:, j])
    return c


In [7]:
test_close(t1, matmul(m1, m2), eps=2e-5)


PyTorch tensors on CPU with builtin dot product is still slightly slower than `njit`, though better than the ad-hoc dot product.


In [8]:
%timeit -n 50 _ = matmul(m1, m2)


477 µs ± 14.6 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)
